In [1]:
import random
import tensorflow_addons as tfa
from WarmUpCosine import *
from util import *
from missing_modality.model import create_vit_classifier

c:\python\python38\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## CIFAR-10 dataset loading and preparation

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
(x_train, y_train), (x_val, y_val) = ((x_train[:40000], y_train[:40000]),(x_train[40000:], y_train[40000:]),)
train_ds = prepare_data(x_train, y_train)
val_ds = prepare_data(x_train, y_train, is_train=False)
test_ds = prepare_data(x_test, y_test, is_train=False)

## Train and evaluate model

In [3]:
total_steps = int((len(x_train) / BATCH_SIZE) * EPOCHS)
warmup_steps = int(total_steps * 0.15)
scheduled_lrs = WarmUpCosine(learning_rate_base=LEARNING_RATE, total_steps=total_steps, warmup_learning_rate=0.0, warmup_steps=warmup_steps,)

optimizer = tfa.optimizers.AdamW(learning_rate=scheduled_lrs,weight_decay=WEIGHT_DECAY)
vit_model = create_vit_classifier()
vit_model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])
vit_model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

loss, accuracy = vit_model.evaluate(test_ds)
accuracy = round(accuracy * 100, 2)
print(f"Accuracy on the test set: {accuracy}%.")

Epoch 1/10


InternalError: Graph execution error:

Detected at node 'model/patch_encoder_cls/dense/Tensordot/MatMul' defined at (most recent call last):
    File "c:\python\python38\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\python\python38\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\python\python38\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\python\python38\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "c:\python\python38\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "c:\python\python38\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "c:\python\python38\lib\asyncio\base_events.py", line 570, in run_forever
      self._run_once()
    File "c:\python\python

In [ ]:
vit_model.save(f"classification_vit_model@acc_{accuracy}", include_optimizer=False)